Project By:Ammad_092_C | AI Semester Project Fall 2020


## Install dependancies


In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pyspellchecker
!pip install textblob

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Import dependancies


In [2]:
import io
import spacy
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import re
from time import sleep
from tqdm.notebook import tqdm

## Importing and assembling data




In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_dataset_raw_path = "/content/drive/My Drive/ai-proj-dataset/train_14k_split_conll.txt"
test_dataset_unlabeled_path =  "/content/drive/My Drive/ai-proj-dataset/test_labels_hinglish.txt"
test_dataset_labels_path = "/content/drive/My Drive/ai-proj-dataset/Hindi_test_unalbelled_conll_updated.txt"

In [5]:
# train_dataset_raw = open(train_dataset_raw_path, "r")
# test_dataset_unlabeled =  open(test_dataset_unlabeled_path, "r")
# test_dataset_labels = open(test_dataset_labels_path, "r")

In [6]:
# from textblob import TextBlob

# def fixSpellingEng(word):
#   blob_word = TextBlob(word)
#   return blob_word.correct()

# from pattern.en import suggest

# def fixSpellingEng(word):
#   def reduce_lengthening(text):
#     pattern = re.compile(r"(.)\1{2,}")
#     return pattern.sub(r"\1\1", text)
#   word_wlf = reduce_lengthening(word)
#   return suggest(word_wlf)

from spellchecker import SpellChecker

def fixSpellingEng(word):
  # spell = SpellChecker()
  # return str(spell.correction(word))
  return word

def fixSpellingHin(word):
  return word

### Train dataset


In [7]:
# df = [id,sentence,sentiment]
def constructTrainDf(path):
  count = 0
  df_data = []
  df_row = []
  train_dataset_raw_list = []
  with open(path) as FileObj:
    for line in tqdm(FileObj):
        train_dataset_raw_list.append(' '.join([line.lower()]).split())
  for i in tqdm(range(len(train_dataset_raw_list))):
    if len(train_dataset_raw_list[i]) == 3:
      df_row.append(train_dataset_raw_list[i][1])
      df_row.append('')
      df_row.append(train_dataset_raw_list[i][2])
    elif len(train_dataset_raw_list[i]) == 2:
      if train_dataset_raw_list[i][1] == 'eng':
        df_row[1] += fixSpellingEng(train_dataset_raw_list[i][0]) + ' '
      elif train_dataset_raw_list[i][1] == 'hin':
        df_row[1] += fixSpellingHin(train_dataset_raw_list[i][0]) + ' '
      else:
        pass
    elif len(train_dataset_raw_list[i]) == 0:
      df_data.append(df_row)
      df_row = []
    else:
      pass
  return df_data
  
dataset_df = pd.DataFrame(constructTrainDf(train_dataset_raw_path), columns = ['ID', 'Sentence', 'Sentiment'])

In [8]:
dataset_df

,ID,Sentence,Sentiment
0,4330,nen vist bolest vztek smutek zmatek osam lost ...,neutral
1,41616,nehantics haan yaar neha kab karega woh post u...,neutral
2,6648,rahulgandhi television media congress ke liye ...,negative
3,2512,amitshah narendramodi all india me nrc lagu ka...,positive
4,610,nehr who typomantri anjanaomkashyap pagal hai ...,neutral
...,...,...,...
13995,31686,anandk2012 railminindia irctcofficial piyushgo...,negative
13996,34552,so i couldn't get no one to cover me for my bi...,positive
13997,16924,rt theskindoctor13 najimkhan07 aap logo ki baa...,positive
13998,5556,priyaverma ashutosh83b jay jay shree ram ram r...,positive


In [9]:
dataset_df.to_csv('/content/drive/My Drive/ai-proj-dataset/train_dataset_df.csv')

### Test dataset

In [10]:
# df = [id,sentence]
def constructTestDf(path_x, path_labels):
  df_data = []
  df_row = []
  train_dataset_raw_list = []
  with open(path_x) as FileObj:
    for line in tqdm(FileObj):
        train_dataset_raw_list.append(' '.join([line.lower()]).split())
  print(train_dataset_raw_list[:10])
  for i in tqdm(range(0,len(train_dataset_raw_list))):
    if len(train_dataset_raw_list[i]) == 2:
      if train_dataset_raw_list[i][0] == 'meta':
        df_row.append(train_dataset_raw_list[i][1])
        df_row.append('')
      elif train_dataset_raw_list[i][1] == 'eng':
        df_row[1] += fixSpellingEng(train_dataset_raw_list[i][0]) + ' '
      elif train_dataset_raw_list[i][1] == 'hin':
        df_row[1] += fixSpellingHin(train_dataset_raw_list[i][0]) + ' '
      else:
        pass
    elif len(train_dataset_raw_list[i]) == 0:
      df_data.append(df_row)
      df_row = []
    else:
      pass

  test_dataset_df = pd.DataFrame(df_data, columns = ['ID', 'Sentence'])
  
  dfl_data = []
  label_dataset_raw_list = []  
  with open(path_labels) as FileObj:
    for line in tqdm(FileObj):
      line = line.lower().replace('\n','')
      label_dataset_raw_list.append(' '.join([line]).split(','))

  print(label_dataset_raw_list[:10])
  dfl_data = label_dataset_raw_list[1:]
  test_label_df = pd.DataFrame(dfl_data, columns = ['ID', 'Sentiment'])
  
  return [test_dataset_df,test_label_df]

test_datasets = constructTestDf(test_dataset_labels_path, test_dataset_unlabeled_path)
test_dataset_X_df = test_datasets[0]
test_dataset_Y_df = test_datasets[1]


[['meta', '20803'], ['@', 'o'], ['454dkhan', 'eng'], ['@', 'o'], ['heisunberg', 'eng'], ['_', 'o'], ['agr', 'eng'], ['kse', 'hin'], ['ko', 'hin'], ['itni', 'hin']]



[['uid', 'sentiment'], ['20803', 'neutral'], ['20187', 'neutral'], ['20953', 'neutral'], ['13777', 'negative'], ['20990', 'positive'], ['15075', 'neutral'], ['32292', 'neutral'], ['4622', 'positive'], ['22338', 'negative']]


In [11]:
test_dataset_X_df

,ID,Sentence
0,20803,454dkhan heisunberg agr kse ko itni importantc...
1,20187,logon ko alloo pyaz tomator me toh allah pak k...
2,20953,lambaalka wafadaar bane rahane ka nayab tarika...
3,13777,varnishant narendramodi chup bhosdike he has b...
4,20990,rt hardeepspuri praise the great giver who giv...
...,...,...
2994,7026,narendramodi pradhanmantri ji aap ko dusri baa...
2995,10425,i yogesh22 atulreellife haha atulreellife jai ...
2996,14162,you da realest thank you for putting me on thi...
2997,15860,rt nimra khan241 fahad bhaiiiii give us bilal ...


In [12]:
test_dataset_X_df.to_csv('/content/drive/My Drive/ai-proj-dataset/test_dataset_df.csv')
test_dataset_X_df

,ID,Sentence
0,20803,454dkhan heisunberg agr kse ko itni importantc...
1,20187,logon ko alloo pyaz tomator me toh allah pak k...
2,20953,lambaalka wafadaar bane rahane ka nayab tarika...
3,13777,varnishant narendramodi chup bhosdike he has b...
4,20990,rt hardeepspuri praise the great giver who giv...
...,...,...
2994,7026,narendramodi pradhanmantri ji aap ko dusri baa...
2995,10425,i yogesh22 atulreellife haha atulreellife jai ...
2996,14162,you da realest thank you for putting me on thi...
2997,15860,rt nimra khan241 fahad bhaiiiii give us bilal ...


In [13]:
test_dataset_Y_df

,ID,Sentiment
0,20803,neutral
1,20187,neutral
2,20953,neutral
3,13777,negative
4,20990,positive
...,...,...
2995,10425,neutral
2996,14162,neutral
2997,15860,neutral
2998,25435,neutral


In [14]:
test_dataset_Y_df.to_csv('/content/drive/My Drive/ai-proj-dataset/test_label_df.csv')

In [15]:
test_datase_df = test_dataset_X_df.merge(test_dataset_Y_df, how='left')
test_datase_df

,ID,Sentence,Sentiment
0,20803,454dkhan heisunberg agr kse ko itni importantc...,neutral
1,20187,logon ko alloo pyaz tomator me toh allah pak k...,neutral
2,20953,lambaalka wafadaar bane rahane ka nayab tarika...,neutral
3,13777,varnishant narendramodi chup bhosdike he has b...,negative
4,20990,rt hardeepspuri praise the great giver who giv...,positive
...,...,...,...
2994,7026,narendramodi pradhanmantri ji aap ko dusri baa...,positive
2995,10425,i yogesh22 atulreellife haha atulreellife jai ...,neutral
2996,14162,you da realest thank you for putting me on thi...,neutral
2997,15860,rt nimra khan241 fahad bhaiiiii give us bilal ...,neutral


## Preprocessing the data using Spacy

* Tokenizing sentences to break text down into sentences, words, or other units 
* Removing stop words like “if,” “but,” “or,” and so on
* Normalizing words by condensing all forms of a word into a single form
* Vectorizing text by turning the text into a numerical representation for consumption by your classifier

### Define stop words

In [16]:
from  spacy.lang.en.stop_words import STOP_WORDS

stopwords = list(STOP_WORDS)
stopwords[:]

['if',
 'often',
 'noone',
 'latter',
 'what',
 'been',
 'may',
 'see',
 'across',
 'always',
 'made',
 'becoming',
 'mine',
 'anywhere',
 'hereupon',
 'quite',
 'be',
 'both',
 'will',
 'about',
 'beforehand',
 'whereas',
 'throughout',
 'hereafter',
 'only',
 'per',
 'before',
 'formerly',
 "'s",
 'back',
 'now',
 'every',
 'meanwhile',
 'them',
 'us',
 'forty',
 'used',
 'became',
 'former',
 'less',
 'none',
 'this',
 'herein',
 'anyway',
 'who',
 'her',
 'onto',
 'and',
 'although',
 'his',
 'put',
 'full',
 'do',
 'mostly',
 'seemed',
 'could',
 'anyone',
 'done',
 "'d",
 'among',
 'next',
 'nine',
 'perhaps',
 'just',
 'so',
 'nobody',
 'make',
 'whole',
 'twelve',
 'fifteen',
 'was',
 'himself',
 'however',
 'they',
 'had',
 'such',
 'after',
 'otherwise',
 'down',
 'in',
 'several',
 'the',
 '‘ll',
 'it',
 'below',
 'fifty',
 'from',
 'me',
 'i',
 'become',
 'part',
 'there',
 'together',
 'three',
 'where',
 '‘m',
 'whereafter',
 'further',
 'into',
 'beside',
 'more',
 'shou

###Define punctuation

In [17]:
import string

punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Define parser

In [18]:
from spacy.lang.en import English

parser = English()
parser

### Define normalizer


In [19]:
def normalizer(tokens):
  return [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]

### Define tokenizer

In [20]:
def tokenizer(text):
  parsed_tokens = parser(text)
  normalized_tokens = normalizer(parsed_tokens)
  return [ word for word in normalized_tokens if word not in stopwords and word not in punctuations ]

### Define text cleaner

In [21]:
def clean_txt(str):     
  return str.strip().lower()

## Arm Spacy

In [22]:
import spacy
import en_core_web_sm

spacy_nlp = en_core_web_sm.load()

## Train Model




### Define Transformer

In [23]:
from sklearn.base import TransformerMixin 

class cleaner(TransformerMixin):
  def transform(self, X, **transform_params):
    return_list = []
    for text in X:
      return_list.append(clean_txt(text))
    return return_list

  def fit(self, X, y, **fit_params):
    return self

### Define Vectorizer and Tfvectorizer

In [24]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenizer,
    lowercase = True,
    ngram_range=(1, 1),
    )

### Define classifiers to experiment with

In [25]:
from sklearn.svm import SVC

# classifier = SVC(probability=True, kernel="linear", class_weight="balanced")

In [26]:
from sklearn.ensemble import RandomForestClassifier

# classifier = RandomForestClassifier(n_estimators=200, random_state=0)

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

# classifier = GradientBoostingClassifier(random_state=0, n_estimators = 250, learning_rate=0.2) #0.665
# classifier = GradientBoostingClassifier(random_state=0, n_estimators = 300, learning_rate=0.2) #0.668
# classifier = GradientBoostingClassifier(random_state=0, n_estimators = 300, learning_rate=0.25) #0.671
classifier = GradientBoostingClassifier(random_state=0, n_estimators = 320, learning_rate=0.3, max_depth=3, warm_start = True) 

In [28]:
from sklearn.ensemble import AdaBoostClassifier

# classifier = AdaBoostClassifier(n_estimators=50, base_estimator=classifiersvm, learning_rate=1)

### Features and Labels


In [29]:
X = dataset_df['Sentence']
Y_labels = dataset_df['Sentiment']

In [30]:
X_train = dataset_df['Sentence']
Y_train = dataset_df['Sentiment']
X_test = test_datase_df['Sentence']
Y_test = test_datase_df['Sentiment']

### Pipeline

Clean, tokenize, vectorize, and classify using"Count Vectorizor"

In [31]:
from sklearn.pipeline import Pipeline

pipe_countvect = Pipeline([
                           ("cleaner", cleaner()),
                           ('vectorizer', vectorizer),
                           ('classifier', classifier)
                          ])

### Fit data

In [32]:
pipe_countvect.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.cleaner object at 0x7fc0b8f75898>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 toke...
                                            learning_rate=0.3, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_s

## Predict

In [33]:
sentiment_prediction = pipe_countvect.predict(X_test)
Y_pred = sentiment_prediction

## Results

In [34]:
for (sentence,prediction_sentiment) in zip(X_test,sentiment_prediction):
    print(sentence,"Prediction: ",prediction_sentiment)

454dkhan heisunberg agr kse ko itni importantce chaeay ni tou  Prediction:  neutral
logon ko alloo pyaz tomator me toh allah pak ka naam nazar aa jata hai pr aankhon k samne allah pak ke bande nazar https t co hbg7zs0viy  Prediction:  neutral
lambaalka wafadaar bane rahane ka nayab tarika hai lamba ji aap us party se judi hai jiska sardar arvindkejriwal https t co krvtd1zaek  Prediction:  neutral
varnishant narendramodi chup bhosdike he has been exonerated of all the charges tum log ghanta prove nahi kar paaye  Prediction:  negative
rt hardeepspuri praise the great giver who gives sustenance to all paid obeisance at gurudwara sr  Prediction:  neutral
bomanirani hny2 ki announcement hogyi to humka iss gola me nhi rehna hum nikal jaounga dusre gola me  Prediction:  neutral
rt sharmakef we're almost in the final three-days of ramadan very close to the finish line so let's make sure that we double up our i  Prediction:  neutral
6thrat this song makes me super happy i sing it for myself tak

### Accuracy, Precision, Recall, F1

In [35]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 

# X_train = dataset_df['Sentence']
# Y_train = dataset_df['Sentiment']
# X_test = test_datase_df['Sentence']
# Y_test = test_datase_df['Sentiment']
# Y_pred = sentiment_prediction

print("f1_score",f1_score(Y_test, Y_pred, average="macro"))
print("accuracy_score",accuracy_score(Y_test, Y_pred))  
print("precision_score",precision_score(Y_test, Y_pred, average="macro"))
print("recall_score",recall_score(Y_test, Y_pred, average="macro"))    

f1_score 0.670839884348429
accuracy_score 0.6658886295431811
precision_score 0.6895679350819889
recall_score 0.6630372130650217


## Save files

In [36]:
type(Y_pred)

numpy.ndarray

In [37]:
Y_pred.size

2999

In [38]:
type(test_datase_df['ID'].to_numpy())

numpy.ndarray

In [39]:
test_datase_df['ID'].to_numpy().size

2999

In [40]:
uids = test_datase_df['ID']
uids = uids.to_numpy()

In [41]:
!rm /content/drive/MyDrive/ai-proj-dataset/answer.txt
with open('/content/drive/MyDrive/ai-proj-dataset/answer.txt', 'w') as writefile:
    # writefile.write("Uid,Sentiment")
    for uid, yp in zip(uids, Y_pred):
        writefile.write("\n" + uid+ "," + yp)
